In [1]:
import os
os.chdir('/mnt/jw01-aruk-home01/projects/ra_challenge/RA_challenge/RA2_alpine_lads/ra_joint_predictions')

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from utils.config import Config
import PIL
import json
configuration = Config()
tf.config.threading.set_intra_op_parallelism_threads(20)
tf.config.threading.set_inter_op_parallelism_threads(20)

In [3]:
import dataset.combined_joints_dataset as combset
from prediction import joint_damage

In [ ]:
individual_model = keras.models.load_model("/mnt/iusers01/jw01/mdefscs4/ra_challenge/RA_challenge/michael_dev/RA2_alpine_lads/trained_models/v3/feet_narrowing_reg_old_pretrain_adam_shuffle.h5", compile = False)
individual_model_2 = keras.models.Model(individual_model.inputs, individual_model.layers[-14].output) 
individual_model_2.trainable = False


In [ ]:
n_inputs = 6
inputs = []
for i in range(n_inputs):
    inputs.append(keras.layers.Input(shape=(224,224,1)))
outs = []
for i in range(n_inputs):
    outs.append(individual_model_2(inputs[i]))


In [ ]:

pred = keras.layers.Concatenate()(outs)
pred = keras.layers.Dense(256, activation = "relu")(pred)
pred = keras.layers.BatchNormalization()(pred)
pred = keras.layers.Dropout(0.5)(pred)
pred = keras.layers.Dense(256, activation = "relu")(pred)
pred = keras.layers.BatchNormalization()(pred)
pred = keras.layers.Dropout(0.5)(pred)
pred = keras.layers.Dense(1)(pred)

In [ ]:
combined_model = keras.models.Model(inputs=inputs, outputs=[pred])

In [ ]:
combined_model.compile(optimizer = "adam", loss = "mean_squared_error", metrics = ["mae"])

In [ ]:
combined_model

In [ ]:
train_dataset_class = combset.overall_test_feet(configuration,"/mnt/jw01-aruk-home01/projects/ra_challenge/RA_challenge/training_dataset/fixed")
train_data = train_dataset_class.create_generator(outcomes_source = "/mnt/iusers01/jw01/mdefscs4/ra_challenge/RA_challenge/training_dataset/training.csv")

In [ ]:
val_dataset_class = combset.overall_test_feet(configuration,"/mnt/jw01-aruk-home01/projects/ra_challenge/RA_challenge/training_dataset/fixed")
val_data = train_dataset_class.create_generator(outcomes_source = "/mnt/iusers01/jw01/mdefscs4/ra_challenge/RA_challenge/training_dataset/training.csv",joints_source= "./data/predictions/feet_joint_data_test_v2.csv")

In [ ]:
import logging, os

logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"


In [ ]:
combined_model.fit_generator(train_data,validation_data = val_data, epochs = 10, steps_per_epoch = 20, validation_steps = 5)

In [ ]:
for layer in combined_model.layers:
    layer.trainable = True

In [ ]:
combined_model.compile(optimizer = keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False), loss = "mean_squared_error", metrics = ["mae"])

In [ ]:
combined_model.fit_generator(train_data,validation_data = val_data, epochs = 20, steps_per_epoch = 20, validation_steps = 5)

# hands 

In [6]:
individual_model_joints = keras.models.load_model("/mnt/iusers01/jw01/mdefscs4/ra_challenge/RA_challenge/michael_dev/RA2_alpine_lads/trained_models/v3/hands_narrowing_reg_old_pretrain_adam_shuffle.h5", compile = False)
individual_model_joints_no_dense = keras.models.Model(individual_model_joints.inputs, individual_model_joints.layers[-14].output) 
individual_model_joints_no_dense.trainable = False
individual_model_wrist = keras.models.load_model("/mnt/iusers01/jw01/mdefscs4/ra_challenge/RA_challenge/michael_dev/RA2_alpine_lads/trained_models/v3/wrists_narrowing_reg_old_pretrain_adam_shuffle.h5", compile = False)
individual_model_wrist_no_dense = keras.models.Model(individual_model_wrist.inputs, individual_model_wrist.layers[-14].output) 
individual_model_wrist_no_dense.trainable = False

In [7]:

inputs = []
for i in range(11):
    inputs.append(keras.layers.Input(shape=(224,224,1)))
outs = []
for i in range(10):
    outs.append(individual_model_joints_no_dense(inputs[i]))
outs.append(individual_model_wrist_no_dense(inputs[10]))

pred = keras.layers.Concatenate()(outs)
pred = keras.layers.Dense(256, activation = "relu")(pred)
pred = keras.layers.BatchNormalization()(pred)
pred = keras.layers.Dropout(0.5)(pred)
pred = keras.layers.Dense(256, activation = "relu")(pred)
pred = keras.layers.BatchNormalization()(pred)
pred = keras.layers.Dropout(0.5)(pred)
pred = keras.layers.Dense(1)(pred)

In [8]:
combined_model = keras.models.Model(inputs=inputs, outputs=[pred])

In [9]:
combined_model.compile(optimizer = "adam", loss = "mean_squared_error", metrics = ["mae"])

In [10]:
combined_model

In [11]:
train_dataset_class = combset.overall_test_hand(configuration,"/mnt/jw01-aruk-home01/projects/ra_challenge/RA_challenge/training_dataset/fixed")
train_data = train_dataset_class.create_generator(outcomes_source = "/mnt/iusers01/jw01/mdefscs4/ra_challenge/RA_challenge/training_dataset/training.csv")

In [16]:
val_dataset_class = combset.overall_test_hand(configuration,"/mnt/jw01-aruk-home01/projects/ra_challenge/RA_challenge/training_dataset/fixed")
val_data = train_dataset_class.create_generator(outcomes_source = "/mnt/iusers01/jw01/mdefscs4/ra_challenge/RA_challenge/training_dataset/training.csv",joints_source= "./data/predictions/hand_joint_data_test_v2.csv")

In [13]:
import logging, os

logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"


In [ ]:
combined_model.fit_generator(train_data,validation_data = val_data, epochs = 10, steps_per_epoch = 20, validation_steps = 5)

In [18]:
for layer in combined_model.layers:
    layer.trainable = True

In [19]:
combined_model.compile(optimizer = keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False), loss = "mean_squared_error", metrics = ["mae"])

In [ ]:
combined_model.fit_generator(train_data,validation_data = val_data, epochs = 20, steps_per_epoch = 20, validation_steps = 5)

Epoch 1/20
 3/20 [===>..........................] - ETA: 5:40 - loss: 60.1988 - mae: 6.3185

In [9]:
feet_model = keras.models.load_model("weights/SC1_v3A_feet_erosion_mae_adamwafter_model_100.h5", compile = False)

In [10]:
lr_multipliers = {}
for layer in feet_model.layers[:-7]:
    print(layer)
    lr_multipliers.update({layer.name: 0.1})

In [15]:
feet_model.layers[6].layers[1].name

'conv_1_conv_1'